In [2]:
import pandas as pd
import spacy
from spacy.tokenizer import Tokenizer
!python -m spacy download pt_core_news_sm
import pt_core_news_sm
from tqdm import tqdm

sp = pt_core_news_sm.load()

✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')


In [5]:
import os.path

def create_dataset():
    # Abre o arquivo de leis, para formata-lo igual o CRFtoNER
    with open('Datasets/tweets_by_id.txt', 'r') as f_tweets, \
         open('Datasets/ptbr_tweets.csv', 'w+') as file:
        
        # Abre o arquivo dos NET Tweets em ordem decrescente
        df = pd.read_csv('Datasets/dataset.ptbr_tweets.txt', sep="\t")
        df = df.sort_values('tweet_id')
        
        # Abre todas as linhas do arquivo
        tweets = f_tweets.read().split("|TWEET_ID|:")
        
        # Escreve o header do csv
        file.write("Sentence #\tWord\tPOS\tTag\n")

        list_sent = []

        # Percorre linha por linha
        sent_i = 1
        for i, tweet in enumerate(tqdm(tweets)):
            # Pula o cabeçalho
            if i == 0:
                continue
            
            # Pega o ID e a Mensagem
            tweet_id, original = tweet.split("|MENSAGEM|: ")
            
            if "|SEM INFORMAÇÃO|" in original:
                continue
            
            # Modificando a string
            msg = original.replace("'", " ")
            msg = msg.replace('"', ' ')
            msg = msg.replace("“", ' ')
            msg = msg.replace("”", ' ')
            msg = msg.replace(",", " , ")
            msg = msg.replace(";", " ; ")
            msg = msg.replace("-", " - ")
            msg = msg.replace("\n", " ")
            msg = msg.replace("\t", " ")
            
            # Passa pelo Spacy a mensagem
            msg_sp = sp(u"%s" % msg)
            #msg_sp = sp(msg
            
            # Retorna um df apenas com os tweets com essa id
            aux_df = df.loc[df['tweet_id'] == int(tweet_id)]
            
            # Faz uma lista com a
            # (PALAVRA, POSTAG, TAG)
            list_sent = []
            for s in msg_sp:
                list_sent.append([
                    s.text,
                    s.pos_,
                    'O'
                ])
                
            # Conjunto de palavras
            words = [s.text for s in msg_sp]
            
            # Percorre todas as linhas do DF com o mesmo ID
            for index, row in aux_df.iterrows():
                
                # Se for diferente de "No entities found in this tweet",
                # Significa que Possui uma entidade neste tweet
                if row['named_entity_type'] != 'No entities found in this tweet':
                    
                    # Tag da entidade
                    tag = row['named_entity_type']
                    
                    # Posição da entidade no texto
                    begin = row['start_pos'] -1
                    end = row['end_pos'] -1
                    
                    # Devido alguns erros,
                    # a palavra pode estar nessas 3 
                    # posições
                    palavras = [
                        original[begin:end], 
                        original[(begin+1):(end+1)],
                        original[(begin+2):(end+2)],
                    ]    
                    
                    # Percorre todas as palavras
                    found = False
                    for palavra in palavras:
                        
                        # Percorre todas as words da sentença
                        for count, word in enumerate(words):
                            
                            # Se deu match, atualiza o Tag
                            if palavra == word:
                                list_sent[count][2] = tag
                                found = True
                                break
                                
                        # Se encontrou, sai do laço
                        if found:
                            break
            
            for sent in list_sent:
                if sent[0] != ' ':
                    file.write("Sentence %d\t" % sent_i)
                    file.write("%s\t" % sent[0])
                    file.write("%s\t" % sent[1])
                    file.write("%s\n" % sent[2])    
            
            sent_i += 1
            
            
if not os.path.isfile('Datasets/ptbr_tweets.csv'):
    print("Criando dataset...")
    create_dataset()

create_dataset()
print("Dataset criado!")

100%|██████████| 3157/3157 [00:37<00:00, 84.45it/s]

Dataset criado!
